Package Installations

In [1]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch as t

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import os 

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [2]:
!pip install kaggle

In [3]:
!pip install transformers

In [4]:
!pip install -U torchtext==0.8.0

In [5]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Kaggle Dataset

In [8]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
replace Fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Fake.csv                
  inflating: True.csv                


In [60]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

In [61]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [62]:
corpus["x"] = corpus["title"]

In [63]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

In [64]:
corpus = corpus[["x", "y"]]

In [65]:
corpus.drop_duplicates(inplace=True)
corpus

,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


**Reddit Dataset**

In [66]:
reddit_df = pd.read_csv('/content/news_posts.csv')
reddit_df

,Unnamed: 0,Title,Post Text,ID,Score,Total Comments,Post URL
0,0,People Are Accusing Robinhood Of Stealing From...,NaN,l7afyx,181017,4408,https://www.buzzfeednews.com/article/clarissaj...
1,1,US Military Could Lose Space Force Trademark t...,NaN,gyzw2p,129257,2844,https://www.cbr.com/us-military-lose-space-for...
2,2,White House threatens to fire anyone who tries...,NaN,jrskag,126468,4141,https://americanindependent.com/white-house-th...
3,3,Meta's threat to close down Facebook and Insta...,NaN,so0ree,126258,3786,https://www.cityam.com/metas-threat-to-close-d...
4,4,Don't eat or inject yourself with disinfectant...,NaN,g6zci5,125437,7024,https://www.cnn.com/world/live-news/coronaviru...
...,...,...,...,...,...,...,...
4898,987,Colorado pizza delivery driver saves mans life...,NaN,3b37gf,24221,898,http://www.postindependent.com/news/16943384-1...
4899,988,"Utah Woman Donates Over 1,200 Handmade Toys to...",NaN,90pbng,24216,888,https://www.insideedition.com/utah-woman-donat...
4900,989,Year of the Tiger marks increase in tiger popu...,NaN,sinyqg,24197,205,https://democratic-europe.eu/2022/02/01/%ef%bf...
4901,990,Florida cops deliver dresses made by a 99-year...,NaN,8q8njr,24164,183,http://www.miamiherald.com/news/state/florida/...


In [67]:
reddit_df['x'] = reddit_df['Title']

In [68]:
reddit_df['y'] = ["True"]*4903
reddit_df['y'] = y_encoder.transform(reddit_df['y'])

In [69]:
reddit_df = reddit_df[["x", "y"]]
reddit_df.drop_duplicates(inplace=True)
reddit_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,People Are Accusing Robinhood Of Stealing From...,1
1,US Military Could Lose Space Force Trademark t...,1
2,White House threatens to fire anyone who tries...,1
3,Meta's threat to close down Facebook and Insta...,1
4,Don't eat or inject yourself with disinfectant...,1
...,...,...
4898,Colorado pizza delivery driver saves mans life...,1
4899,"Utah Woman Donates Over 1,200 Handmade Toys to...",1
4900,Year of the Tiger marks increase in tiger popu...,1
4901,Florida cops deliver dresses made by a 99-year...,1


In [70]:
#Convert text to lowercase
corpus['x'] = [text.lower() for text in corpus['x']]

#Tokenization
corpus['x'] = [word_tokenize(text) for text in corpus['x']]

#WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [71]:
membersProcessed = 0
for idx, text in enumerate(corpus['x']):
  finalWords = []
  word_net_lemmatizer = WordNetLemmatizer()
  set_stop = set(stopwords.words('english'))
  iterate = pos_tag(text)
  [finalWords.append(word_net_lemmatizer.lemmatize(word, tag_map[tag[0]])) for word, tag in iterate if word not in set_stop and word.isalpha()]
  corpus.loc[idx, 'finalText'] = str(finalWords)
  membersProcessed+=1
  print('Progress: {}/{} members processed'.format(membersProcessed, len(corpus)))

Streaming output truncated to the last 5000 lines.
Progress: 33730/44416 members processed
Progress: 33731/44416 members processed
Progress: 33732/44416 members processed
Progress: 33733/44416 members processed
Progress: 33734/44416 members processed
Progress: 33735/44416 members processed
Progress: 33736/44416 members processed
Progress: 33737/44416 members processed
Progress: 33738/44416 members processed
Progress: 33739/44416 members processed
Progress: 33740/44416 members processed
Progress: 33741/44416 members processed
Progress: 33742/44416 members processed
Progress: 33743/44416 members processed
Progress: 33744/44416 members processed
Progress: 33745/44416 members processed
Progress: 33746/44416 members processed
Progress: 33747/44416 members processed
Progress: 33748/44416 members processed
Progress: 33749/44416 members processed
Progress: 33750/44416 members processed
Progress: 33751/44416 members processed
Progress: 33752/44416 members processed
Progress: 33753/44416 members

In [72]:
#Convert text to lowercase
reddit_df['x'] = [text.lower() for text in reddit_df['x']]

#Tokenization
reddit_df['x'] = [word_tokenize(text) for text in reddit_df['x']]

#WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [73]:
membersProcessed = 0
for idx, text in enumerate(reddit_df['x']):
  finalWords = []
  word_net_lemmatizer = WordNetLemmatizer()
  set_stop = set(stopwords.words('english'))
  iterate = pos_tag(text)
  [finalWords.append(word_net_lemmatizer.lemmatize(word, tag_map[tag[0]])) for word, tag in iterate if word not in set_stop and word.isalpha()]
  reddit_df.loc[idx, 'finalText1'] = str(finalWords)
  membersProcessed+=1
  print('Progress: {}/{} members processed'.format(membersProcessed, len(reddit_df)))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Progress: 1/4903 members processed
Progress: 2/4903 members processed
Progress: 3/4903 members processed
Progress: 4/4903 members processed
Progress: 5/4903 members processed
Progress: 6/4903 members processed
Progress: 7/4903 members processed
Progress: 8/4903 members processed
Progress: 9/4903 members processed
Progress: 10/4903 members processed
Progress: 11/4903 members processed
Progress: 12/4903 members processed
Progress: 13/4903 members processed
Progress: 14/4903 members processed
Progress: 15/4903 members processed
Progress: 16/4903 members processed
Progress: 17/4903 members processed
Progress: 18/4903 members processed
Progress: 19/4903 members processed
Progress: 20/4903 members processed
Progress: 21/4903 members processed
Progress: 22/4903 members processed
Progress: 23/4903 members processed
Progress: 24/4903 members processed
Progress: 25/4903 members processed
Progress: 26/4903 members processed
Progress: 27/4903 members processed
Progress: 28/4903 members processed
P

In [77]:
corpus

,x,y,finalText
0,"[donald, trump, sends, out, embarrassing, new,...",0.0,"['donald', 'trump', 'send', 'embarrass', 'new'..."
1,"[drunk, bragging, trump, staffer, started, rus...",0.0,"['drunk', 'brag', 'trump', 'staffer', 'start',..."
2,"[sheriff, david, clarke, becomes, an, internet...",0.0,"['sheriff', 'david', 'clarke', 'become', 'inte..."
3,"[trump, is, so, obsessed, he, even, has, obama...",0.0,"['trump', 'obsessed', 'even', 'obama', 'name',..."
4,"[pope, francis, just, called, out, donald, tru...",0.0,"['pope', 'francis', 'call', 'donald', 'trump',..."
...,...,...,...
38724,"[war, crimes, court, backs, burundi, investiga...",1.0,"['north', 'korea', 'shipment', 'syria', 'chemi..."
38725,"[uganda, 's, president, boosts, military, unit...",1.0,"['lexisnexis', 'withdraw', 'two', 'product', '..."
38726,"[despite, nato, pledge, to, increase, afghan, ...",1.0,"['minsk', 'cultural', 'hub', 'becomes', 'autho..."
38727,"[syria, declares, victory, over, islamic, state]",1.0,"['vatican', 'upbeat', 'possibility', 'pope', '..."


In [75]:
corpus = corpus.dropna()

In [76]:
corpus

,x,y,finalText
0,"[donald, trump, sends, out, embarrassing, new,...",0.0,"['donald', 'trump', 'send', 'embarrass', 'new'..."
1,"[drunk, bragging, trump, staffer, started, rus...",0.0,"['drunk', 'brag', 'trump', 'staffer', 'start',..."
2,"[sheriff, david, clarke, becomes, an, internet...",0.0,"['sheriff', 'david', 'clarke', 'become', 'inte..."
3,"[trump, is, so, obsessed, he, even, has, obama...",0.0,"['trump', 'obsessed', 'even', 'obama', 'name',..."
4,"[pope, francis, just, called, out, donald, tru...",0.0,"['pope', 'francis', 'call', 'donald', 'trump',..."
...,...,...,...
38724,"[war, crimes, court, backs, burundi, investiga...",1.0,"['north', 'korea', 'shipment', 'syria', 'chemi..."
38725,"[uganda, 's, president, boosts, military, unit...",1.0,"['lexisnexis', 'withdraw', 'two', 'product', '..."
38726,"[despite, nato, pledge, to, increase, afghan, ...",1.0,"['minsk', 'cultural', 'hub', 'becomes', 'autho..."
38727,"[syria, declares, victory, over, islamic, state]",1.0,"['vatican', 'upbeat', 'possibility', 'pope', '..."


In [78]:
reddit_df

,x,y,finalText1
0,"[people, are, accusing, robinhood, of, stealin...",1,"['people', 'accuse', 'robinhood', 'steal', 'po..."
1,"[us, military, could, lose, space, force, trad...",1,"['u', 'military', 'could', 'lose', 'space', 'f..."
2,"[white, house, threatens, to, fire, anyone, wh...",1,"['white', 'house', 'threaten', 'fire', 'anyone..."
3,"[meta, 's, threat, to, close, down, facebook, ...",1,"['meta', 'threat', 'close', 'facebook', 'insta..."
4,"[do, n't, eat, or, inject, yourself, with, dis...",1,"['eat', 'inject', 'disinfectant', 'warn', 'fda..."
...,...,...,...
4898,"[colorado, pizza, delivery, driver, saves, man...",1,"['colorado', 'pizza', 'delivery', 'driver', 's..."
4899,"[utah, woman, donates, over, 1,200, handmade, ...",1,"['utah', 'woman', 'donate', 'handmade', 'toy',..."
4900,"[year, of, the, tiger, marks, increase, in, ti...",1,"['year', 'tiger', 'mark', 'increase', 'tiger',..."
4901,"[florida, cops, deliver, dresses, made, by, a,...",1,"['florida', 'cop', 'deliver', 'dress', 'make',..."


In [79]:
from sklearn.model_selection import KFold

In [80]:
kf = KFold(n_splits=10)

In [81]:
X_train = corpus['finalText']
X_test = reddit_df['finalText1']
y_train = corpus['y']
y_test = reddit_df['y']

In [84]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [86]:
acc_ls = []
TFIDF_vect = TfidfVectorizer(max_features=5000)
x_train_tfidf = TFIDF_vect.fit_transform(X_train)
x_test_tfidf = TFIDF_vect.transform(X_test)
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train_tfidf,y_train)
predictions_NB = Naive.predict(x_test_tfidf)
acc = accuracy_score(predictions_NB, y_test)
acc_ls.append(acc)
  
print("Naive Bayes Accuracy Score " + " -> ",acc*100)
print("Mean Accuracy : ", sum(acc_ls)*100/len(acc_ls))

Naive Bayes Accuracy Score  ->  35.5904548235774
Mean Accuracy :  35.5904548235774


In [87]:
acc_ls = []
TFIDF_vect = TfidfVectorizer(max_features=5000)
x_train_tfidf = TFIDF_vect.fit_transform(X_train)
x_test_tfidf = TFIDF_vect.transform(X_test)
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train_tfidf, y_train)
predictions_SVM = SVM.predict(x_test_tfidf)
acc = accuracy_score(predictions_SVM, y_test)
acc_ls.append(acc)
print("SVM Accuracy Score " + " -> ",acc*100)
print("Mean Accuracy : ", sum(acc_ls)*100/len(acc_ls))

SVM Accuracy Score  ->  46.13501937589231
Mean Accuracy :  46.13501937589231
